In [1]:
from sage.all import *
# Setup of finite field and polynomial ring
m = 3
t = 2
F = GF(2**m, 'z')
R = PolynomialRing(F, 'x')
x = R.gen()
z = F.gen()
Polynomial = z^2*x^2 + (z^2 + 1)*x + z^2 + 1
Support = sorted([a for a in F])
ciphertext = [1, 1, 1, 1, 0, 1, 0, 0]

# Inverse computation function
def compute_inverse(value):
    try:
        inverse = value.inverse_mod(Polynomial)
        verify = (inverse * value) % Polynomial
        if verify == 1:
            return inverse, True
    except ZeroDivisionError:
        return None, False

# Function to compute rho
def compute_rho(value):
    return ((value + x) % Polynomial)^(2^(m*t - 1)) % Polynomial

# Function to find polynomials a(x) and b(x)
def finding_ax_bx():
    bx = R(1)  # choosing b(x)
    Sx_inverse, _ = compute_inverse(Sx)
    rho_Sx_inverse = compute_rho(Sx_inverse)
    
    ax = (bx * rho_Sx_inverse) % Polynomial
    if ax.degree() <= floor(t):
        #print(f"a(x) satisfied: {ax}")
        return ax, bx
    else:
        print(f"a(x) not satisfied.")
        return None, None

# Function to compute sigma
def sigma(ax, bx):
    return (ax^2) + x*(bx^2)

# Function to find roots of sigma
def root_sigma():
    polinomio = (z^2 + z)*x^2 + x + z^2 + z
    roots = polinomio.roots(multiplicities=False)
    error_vector = [0]*len(Support)
    for i, root in enumerate(roots):
        if root[0] in Support:
            error_vector[Support.index(root[0])] = 1
    return error_vector

# Function to compute the corrected codeword
def find_codeword(ciphertext, e):
    return [int(a != b) for a, b in zip(ciphertext, e)]

# Compute list of syndromes
def syndromes(ciphertext):
    List_Syndrome = [x - Support[i] for i, bit in enumerate(ciphertext) if bit != 0]
    return List_Syndrome

# Main calculations
List_Sx = [x, (x - 1), (x - z), (x - (z^2 + z))]
List = []

for item in List_Sx:
    aux, _ = compute_inverse(item)
    List.append(aux)
Sx = sum(List)
print("sx: ", Sx, "\n\n" )

Sx_inverse, _ = compute_inverse(Sx)
ax, bx = finding_ax_bx()


print("ax   =  ",ax, "\t bx=", bx)
print( "sigma  = ",sigma(ax, bx))
error = root_sigma()
print("error :   ",error)
codeword = find_codeword(ciphertext, error)
print("codeword :", codeword)
print(syndromes(ciphertext))
print()

sx:  (z + 1)*x + z^2 + 1 


ax   =   z^2*x + z^2 	 bx= 1
sigma  =  (z^2 + z)*x^2 + x + z^2 + z
error :    [1, 1, 0, 0, 0, 0, 0, 0]
codeword : [0, 0, 1, 1, 0, 1, 0, 0]
[x, x + 1, x + z, x + z + 1, x + z^2 + 1]
